In [108]:
%reload_ext autoreload
%autoreload 2

from pathlib import Path

import pandas as pd

# Data

In [109]:
data_dir = Path('T:/data/base_2022/CVM/Calib_CVM_B/')
data_dir

WindowsPath('T:/data/base_2022/CVM/Calib_CVM_B')

In [110]:
cvm_zonal_properties = data_dir/'model_output/Zonal Properties CVM.csv'

employment = (
    pd.read_csv(
        cvm_zonal_properties,
        usecols=[
            'TAZ', 'CVM_IN', 'CVM_RE', 'CVM_SV', 'CVM_TH', 'CVM_WH', 'CVM_WH', 'CVM_GO',
        ],
    )
    .rename(
        columns={
            'CVM_IN': 'Industrial',
            'CVM_RE': 'Retail',
            'CVM_SV': 'Service',
            'CVM_TH': 'Transport',
            'CVM_WH': 'Wholesale',
            'CVM_GO': 'Government/Office',
        }
    )
    .melt(id_vars='TAZ', var_name='industry', value_name='employment')
    .assign(
        industry=lambda df: (
            df.industry
            .map(
                {
                    'Industrial': 'Industrial',
                    'Retail': 'Retail',
                    'Service': 'Service/Gov/Office/FA',
                    'Transport': 'Transport',
                    'Wholesale': 'Wholesale',
                    'Government/Office': 'Service/Gov/Office/FA',
                }
            )
        )
    )
    .set_index('TAZ')
    .groupby('industry')
    .sum()
)
employment

,employment
industry,
Industrial,207356.0
Retail,144185.0
Service/Gov/Office/FA,1066343.0
Transport,37037.0
Wholesale,43668.0


In [111]:
cvm_tours_path = data_dir/'model_output/commercialVehicleTours.csv'

tours = (
    pd.read_csv(
        cvm_tours_path,
        usecols=['tourID', 'actorType', 'departTimeFiveTod', 'tourMode'],
    )
    .assign(
        industry=lambda df: df.actorType
        .map(
            {
                'Government\\Office': 'Service/Gov/Office/FA',
                'Service': 'Service/Gov/Office/FA',
                'Fleet Allocator': 'Service/Gov/Office/FA',
                'Industry': 'Industrial',
                'Retail': 'Retail',
                'Transport': 'Transport',
                'Wholesale': 'Wholesale',
            }
        )
    )
    .assign(
        tod=lambda df: df.departTimeFiveTod
        .map(
            {
                1: 'OE (3am - 6am)',
                2: 'AM (6am - 9am)',
                3: 'MD (9am - 3:30pm)',
                4: 'PM (3:30pm - 7pm)',
                5: 'OL (7pm - 3am)',
            }
        )
        .astype(
            pd.CategoricalDtype(
                categories=[
                    'OE (3am - 6am)',
                    'AM (6am - 9am)',
                    'MD (9am - 3:30pm)',
                    'PM (3:30pm - 7pm)',
                    'OL (7pm - 3am)',
                ],
                ordered=True,
            )
        )
    )
    .assign(
        mode=lambda df: df.tourMode
        .map(
            {
                'Drive Alone': 'DA',
                'Light Heavy Duty Truck': 'LHD',
                'Medium Heavy Duty Truck': 'MHD',
                'Heavy Heavy Duty Truck': 'HHD',
            }
        )
        .astype(
            pd.CategoricalDtype(
                categories=[
                    'DA',
                    'LHD',
                    'MHD',
                    'HHD',
                ],
                ordered=True,
            )
        )
    )
    .drop(columns=['actorType', 'departTimeFiveTod', 'tourMode'])
    .set_index('tourID')
)
tours.head(2)

,industry,tod,mode
tourID,,,
1,Industrial,MD (9am - 3:30pm),LHD
2,Industrial,MD (9am - 3:30pm),DA


In [112]:
cvm_trips_path = data_dir/'model_output/commercialVehicleTrips.csv'

trips = (
    pd.read_csv(
        cvm_trips_path,
        usecols=['tripID', 'tourID', 'stopID', 'weightTrip', 'distanceTotal'],
    )
    .set_index(['tripID', 'tourID', 'stopID'])
)
trips.head(2)

,,,weightTrip,distanceTotal
tripID,tourID,stopID,,
1,1,1,3.36,13.258280
2,1,2,3.36,13.258282


In [113]:
merged = (
    trips.reset_index()
    .merge(tours.reset_index(), on='tourID', how='left')
    .set_index(['tripID', 'tourID', 'stopID'])
    .assign(vmt=lambda df: df.weightTrip * df.distanceTotal)
)
merged.head(2)

,,,weightTrip,distanceTotal,industry,tod,mode,vmt
tripID,tourID,stopID,,,,,,
1,1,1,3.36,13.258280,Industrial,MD (9am - 3:30pm),LHD,44.547821
2,1,2,3.36,13.258282,Industrial,MD (9am - 3:30pm),LHD,44.547828


# Summaries

### Table 15

In [120]:
table_15_data = pd.DataFrame(
    {
        'Employees': employment.groupby('industry').employment.sum(),
        'Tours': merged.groupby(['industry', 'tourID']).weightTrip.max().to_frame().groupby('industry').weightTrip.sum(),
        'Trips': merged.groupby('industry').weightTrip.sum(),
        'Trips_DA': merged.query("`mode` == 'DA'").groupby('industry').weightTrip.sum(),
        'Trips_LHD': merged.query("`mode` == 'LHD'").groupby('industry').weightTrip.sum(),
        'Trips_MHD': merged.query("`mode` == 'MHD'").groupby('industry').weightTrip.sum(),
        'Trips_HHD': merged.query("`mode` == 'HHD'").groupby('industry').weightTrip.sum(),
        'VMT': merged.groupby('industry').vmt.sum(),
        'VMT_DA': merged.query("`mode` == 'DA'").groupby('industry').vmt.sum(),
        'VMT_LHD': merged.query("`mode` == 'LHD'").groupby('industry').vmt.sum(),
        'VMT_MHD': merged.query("`mode` == 'MHD'").groupby('industry').vmt.sum(),
        'VMT_HHD': merged.query("`mode` == 'HHD'").groupby('industry').vmt.sum(),
    }
)
table_15_data.to_csv('T:/data/base_2022/CVM/Calib_CVM_B/model_table15.csv')
table_15_data


,Employees,Tours,Trips,Trips_DA,Trips_LHD,Trips_MHD,Trips_HHD,VMT,VMT_DA,VMT_LHD,VMT_MHD,VMT_HHD
industry,,,,,,,,,,,,
Industrial,207356.0,48660.5,161734.0,5154.84,132392.84,2731.32,21455.0,1.281649e+06,37183.000101,9.815495e+05,30945.561119,231970.876978
Retail,144185.0,38513.5,169507.0,5947.24,156088.20,4102.56,3369.0,1.218760e+06,42250.479137,1.112536e+06,33140.728640,30833.323651
Service/Gov/Office/FA,1066343.0,219446.0,986475.0,29000.88,822746.28,61980.84,72747.0,7.616048e+06,214863.131919,6.196111e+06,517796.231943,687277.408159
Transport,37037.0,10022.0,41896.0,1135.92,31119.64,1621.44,8019.0,2.636991e+05,6647.048247,1.891413e+05,12525.147665,55385.536442
Wholesale,43668.0,9341.0,52166.5,1705.78,46876.28,2215.44,1369.0,5.315402e+05,16773.169017,4.667285e+05,26765.527016,21272.962856


### Figure 9

In [115]:
fig_9_data = ( # The time-of-day distribution of weighted tours for each industry
    (
        ( # Total weighted tours for each industry, for each time of day
            merged.reset_index()
            .drop_duplicates(subset='tourID')
            .groupby(['industry', 'tod', 'tourID']).max()
            .groupby(['industry', 'tod'])
            .weightTrip.sum()
        ) 
        / ( # Total weighted tours for each industry
            merged.reset_index()
            .drop_duplicates(subset='tourID')
            .groupby(['industry', 'tod', 'tourID']).max()
            .groupby(['industry'])
            .weightTrip.sum()
        )
    )
    .to_frame()
    .reset_index()
    .pivot(columns='tod', index='industry', values='weightTrip')
)
fig_9_data.to_csv('T:/data/base_2022/CVM/Calib_CVM_B/model_figure9.csv')
fig_9_data

tod,OE (3am - 6am),AM (6am - 9am),MD (9am - 3:30pm),PM (3:30pm - 7pm),OL (7pm - 3am)
industry,,,,,
Industrial,0.275932,0.426671,0.283485,0.011673,0.002240
Retail,0.117647,0.395550,0.468057,0.015060,0.003687
Service/Gov/Office/FA,0.157178,0.403389,0.423690,0.013324,0.002420
Transport,0.316903,0.435342,0.236081,0.005089,0.006586
Wholesale,0.249224,0.434964,0.305642,0.009314,0.000856


### Figure 10

In [116]:
fig_10_data = ( # The vehicle mode distribution of weighted tours for each industry
    (
        ( # Total weighted tours for each industry, for vehicle mode
            merged.reset_index()
            .drop_duplicates(subset='tourID')
            .groupby(['industry', 'mode', 'tourID']).max()
            .groupby(['industry', 'mode'])
            .weightTrip.sum()
        ) 
        / ( # Total weighted tours for each industry
            merged.reset_index()
            .drop_duplicates(subset='tourID')
            .groupby(['industry', 'mode', 'tourID']).max()
            .groupby(['industry'])
            .weightTrip.sum()
        )
    )
    .to_frame()
)
fig_10_data.to_csv('T:/data/base_2022/CVM/Calib_CVM_B/model_figure10.csv')
fig_10_data

weightTrip
industry              mode            
Industrial            DA      0.031796
                      LHD     0.798103
                      MHD     0.011245
                      HHD     0.158856
Retail                DA      0.034960
                      LHD     0.918401
                      MHD     0.024855
                      HHD     0.021785
Service/Gov/Office/FA DA      0.033316
                      LHD     0.876295
                      MHD     0.035838
                      HHD     0.054551
Transport             DA      0.022275
                      LHD     0.632473
                      MHD     0.041022
                      HHD     0.304231
Wholesale             DA      0.032313
                      LHD     0.878720
                      MHD     0.040043
                      HHD     0.048924

### Figure 12

In [117]:
fig_12_data = ( # The vehicle mode distribution of weighted tours for each industry
    (
        ( # Weighted average trip length for each mode/industry
            (
                merged
                .assign(weighted_distance=lambda df: df.distanceTotal * df.weightTrip)
                .groupby(['mode', 'industry']).weighted_distance.sum()
            )
            / merged.groupby(['mode', 'industry']).weightTrip.sum()
        ) 
    )
    .rename('distance')
    .to_frame()
)
fig_12_data.to_csv('T:/data/base_2022/CVM/Calib_CVM_B/model_figure12.csv')
fig_12_data

distance
mode industry                        
DA   Industrial              7.213221
     Retail                  7.104216
     Service/Gov/Office/FA   7.408849
     Transport               5.851687
     Wholesale               9.833137
LHD  Industrial              7.413917
     Retail                  7.127611
     Service/Gov/Office/FA   7.531011
     Transport               6.077876
     Wholesale               9.956603
MHD  Industrial             11.329892
     Retail                  8.078061
     Service/Gov/Office/FA   8.354134
     Transport               7.724706
     Wholesale              12.081359
HHD  Industrial             10.811973
     Retail                  9.152070
     Service/Gov/Office/FA   9.447502
     Transport               6.906788
     Wholesale              15.539052